In [1]:
pip install numpy==1.22.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
plotnine 0.12.4 requires numpy>=1.23.0, but you have numpy 1.22.3 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 1.22.3 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.22.3 which is incompatible.


In [2]:
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 60.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-

In [3]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.9 MB/s eta 0:00:00


In [5]:
pip install accelerate -U

In [2]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling, TrainerCallback  # Added import for TrainerCallback
from torch.utils.data import Dataset
import re
import torch
import logging
from tqdm.auto import tqdm
import os
#fixing issues with wandb crushing the run
os.environ["WANDB_DISABLED"] = "true"
#callbacks for progress monitoring
class ProgressCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_progress_bar = tqdm(total = state.max_steps, desc = "epoch progress")

    def on_step_end(self, args, state, control, **kwargs):
        self.epoch_progress_bar.update(1)
    def on_epoch_end(self, args, state, control, **kwargs):
        self.epoch_progress_bar.close()
#logging for monitoring the progress
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    level=logging.INFO
)
device = "cuda"
#function preparing the dataset
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size = 512):
        with open(file_path, 'r', encoding="utf-8") as f:
            text = f.read()

        lines = re.findall(r'<st>(.*?)</st>', text)
        self.examples = []
        for line in lines:
            encoding = tokenizer(line, add_special_tokens=True, truncation=True, max_length=block_size, padding='max_length', return_tensors='pt')
            self.examples.append({key: val.squeeze() for key, val in encoding.items()})

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return {"input_ids": self.examples[i]['input_ids'], "attention_mask": self.examples[i]['attention_mask']}

#default tokenizer and distilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
model.cuda()
#setting the transformations
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = True, mlm_probability = 0.15)
#loading and tokenizng the dataset
dataset = TextDataset(tokenizer, "/content/SaveFilteredCorpus.txt")
#params and hyperparameters for training
training_args = TrainingArguments(
    output_dir = "./results",
    overwrite_output_dir = True,
    num_train_epochs = 5,  # Number of epochs
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 4,
    save_steps = 100,
    save_total_limit = 2,
    report_to = None,
    logging_dir = './logs',
    logging_steps = 10,
    fp16 = True,
)
#setting params for trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset,
    callbacks = [ProgressCallback()]
)
#fine-tuning
trainer.train()
#saving the architecture and weights
model.save_pretrained("my_trained_model")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


epoch progress:   0%|          | 0/8335 [00:00<?, ?it/s]

Step,Training Loss
10,3.457500
20,2.911800
30,2.732100
40,2.682700
50,2.524800
60,2.410000
70,2.492700
80,2.263700
90,2.583700
100,2.339300


epoch progress:   0%|          | 0/8335 [00:00<?, ?it/s]

epoch progress:   0%|          | 0/8335 [00:00<?, ?it/s]

epoch progress:   0%|          | 0/8335 [00:00<?, ?it/s]

epoch progress:   0%|          | 0/8335 [00:00<?, ?it/s]

In [56]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch
#saved model
model_dir = "/content/my_trained_model"
#default tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#model loading
model = DistilBertForMaskedLM.from_pretrained(model_dir)
#masked task
text = "Ford makes [MASK] EVs than Tesla"
#tokenizing input
inputs = tokenizer(text, return_tensors = "pt")
#inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
#finding the masked token
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
#getting logits
num_predictions = 10
mask_token_logits = outputs.logits[0, mask_token_index, : ]
top_token_indices = torch.topk(mask_token_logits, num_predictions, dim = 1).indices[0]
#decoding the top token
predicted_tokens = [tokenizer.decode([idx]) for idx in top_token_indices]
#printing predictions
print("top predictions:")
for i, token in enumerate(predicted_tokens, 1):
    print(f"{i}: {token}")

top predictions:
1: more
2: better
3: cheaper
4: faster
5: less
6: worse
7: fewer
8: bigger
9: smaller
10: slower
